In [33]:
import os
import pandas as pd
import pickle
import numpy
import pvdeg

In [10]:
file = r'C:\Users\mbrown2\Documents\GitHub\PVDegradationTools\tests\data\module_temp_day.pkl'
with open(file, 'rb') as p:
    mod_temp_day = pickle.load(p)
mod_temp_day

2021-01-01 00:00:00+00:00   -2.300000
2021-01-01 00:10:00+00:00   -2.400000
2021-01-01 00:20:00+00:00   -2.500000
2021-01-01 00:30:00+00:00   -2.500000
2021-01-01 00:40:00+00:00   -2.600000
                               ...   
2021-01-01 23:10:00+00:00    5.771742
2021-01-01 23:20:00+00:00    4.279285
2021-01-01 23:30:00+00:00    2.727496
2021-01-01 23:40:00+00:00   -0.900000
2021-01-01 23:50:00+00:00   -0.900000
Length: 144, dtype: float64

In [40]:
file = r'C:\Users\mbrown2\Documents\GitHub\PVDegradationTools\tests\data\legacy_data\rel_hum_year.pkl'
with open(file, 'rb') as p:
    demo = pickle.load(p)
demo

,surface_outside,front_encap,back_encap,backsheet
2021-01-01 00:00:00+00:00,56.219998,36.258976,56.219998,56.219998
2021-01-01 00:10:00+00:00,56.639995,36.358597,56.378439,56.509217
2021-01-01 00:20:00+00:00,57.059995,36.458566,56.541289,56.800642
2021-01-01 00:30:00+00:00,57.059995,36.458566,56.549010,56.804503
2021-01-01 00:40:00+00:00,55.939995,36.558883,56.693298,56.316646
...,...,...,...,...
2021-12-31 23:10:00+00:00,68.544973,35.515726,54.477579,61.511276
2021-12-31 23:20:00+00:00,69.352795,35.655949,54.922281,62.137538
2021-12-31 23:30:00+00:00,70.171319,35.796876,55.372679,62.771999
2021-12-31 23:40:00+00:00,71.139997,35.962185,55.866630,63.503314


In [35]:
folder = r'C:\Users\mbrown2\Documents\GitHub\PVDegradationTools\tests\data\legacy_data'

file_buffer = next(os.walk(folder))[2]

d_pickles = [f for f in file_buffer if 'day' in f]
y_pickles = [f for f in file_buffer if 'year' in f]

print(d_pickles)
print(y_pickles)

['module_temp_day.pkl', 'poa_irradiance_day.pkl', 'rel_hum_day.pkl', 'solar_position_day.pkl', 'weather_df_day.pkl']
['module_temp_year.pkl', 'poa_irradiance_year.pkl', 'rel_hum_year.pkl', 'solar_position_year.pkl', 'weather_df_year.pkl']


In [44]:
df = pd.DataFrame()
for yp in y_pickles[:-1]:
    name = yp.split('_')
    name = f'{name[0]}_{name[1]}'
    print(name)
    with open(os.path.join(folder,yp), 'rb') as p:
        vals = pickle.load(p)
    if 'hum' in name:
        for col in vals.columns:
            col_name = f'RH_{col}'
            df[col_name] = vals[col]
    elif 'solar' in name or 'poa' in name:
        for col in vals.columns:
            df[col] = vals[col]
    else:
        df[name] = vals


module_temp
poa_irradiance
rel_hum
solar_position


In [45]:
df.to_csv('input_year_pytest.csv')

In [39]:
year_df_full = pd.concat([weather_year, year_df], axis=1)
year_df_full

,air_temperature,wind_speed,dhi,ghi,dni,relative_humidity,module_temp,poa_global,surface_outside,front_encap,back_encap,backsheet,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
2021-01-01 00:00:00+00:00,-2.3,1.1,0,0,0,56.220001,-2.300000,0.000000,56.219998,36.258976,56.219998,56.219998,93.117688,93.117688,-3.117688,-3.117688,242.378971,-3.431294
2021-01-01 00:10:00+00:00,-2.4,1.1,0,0,0,56.639999,-2.400000,0.000000,56.639995,36.358597,56.378439,56.509217,94.831823,94.831823,-4.831823,-4.831823,243.917008,-3.434562
2021-01-01 00:20:00+00:00,-2.5,1.1,0,0,0,57.060001,-2.500000,0.000000,57.059995,36.458566,56.541289,56.800642,96.568483,96.568483,-6.568483,-6.568483,245.433311,-3.437830
2021-01-01 00:30:00+00:00,-2.5,1.1,0,0,0,57.060001,-2.500000,0.000000,57.059995,36.458566,56.549010,56.804503,98.326156,98.326156,-8.326156,-8.326156,246.930223,-3.441097
2021-01-01 00:40:00+00:00,-2.6,1.1,0,0,0,55.939999,-2.600000,0.000000,55.939995,36.558883,56.693298,56.316646,100.103395,100.103395,-10.103395,-10.103395,248.410135,-3.444365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 23:10:00+00:00,-2.0,2.8,19,19,0,71.139999,-1.599959,17.353793,68.544973,35.515726,54.477579,61.511276,84.850158,84.979218,5.149842,5.020782,234.292213,-3.281420
2021-12-31 23:20:00+00:00,-2.0,2.8,13,13,0,71.139999,-1.726288,11.873648,69.352795,35.655949,54.922281,62.137538,86.387678,86.555556,3.612322,3.444444,235.971842,-3.284718
2021-12-31 23:30:00+00:00,-2.0,2.8,7,7,0,71.139999,-1.852617,6.393503,70.171319,35.796876,55.372679,62.771999,87.927780,88.163118,2.072220,1.836882,237.618704,-3.288016
2021-12-31 23:40:00+00:00,-2.0,2.8,0,0,0,71.139999,-2.000000,0.000000,71.139997,35.962185,55.866630,63.503314,89.434654,89.799993,0.565346,0.200007,239.234918,-3.291314


In [40]:
year_df_full.to_csv('input_year_pytest.csv')

***
# TEST
***

In [6]:
import os
import json
import pandas as pd
import pvdeg 
from pvdeg import TEST_DATA_DIR
from pytest import approx

#Load weather data
WEATHER = pd.read_csv(os.path.join(TEST_DATA_DIR, 'weather_day_pytest.csv'),
                         index_col= 0, parse_dates=True)
def test_psat():
    '''
    test pvdeg.humidity.psat
    
    Requires:
    ---------
    weahter dataframe and meta dictionary
    '''
    psat_avg = pvdeg.humidity.psat(temp=WEATHER['air_temperature'])[1]
    assert psat_avg == approx(0.47607, abs=5e-5)

In [81]:
PSM_NEW = os.path.join('data','psm3_pytest.csv')
PSM_NEW = pd.read_csv(PSM_NEW,header=2,parse_dates=True)
PSM_OLD = os.path.join('data','psm3_pytest_old.csv')
PSM_OLD = pd.read_csv(PSM_OLD, header=2)


In [84]:
PSM_NEW['time_range'] = pd.to_datetime(PSM_NEW['time_range'])


In [87]:
PSM_NEW.index = PSM_NEW['time_range']
temp_cell = PSM_NEW['temp_cell']


time_range
1999-01-01 00:30:00-07:00    0.0
1999-01-01 01:30:00-07:00    0.0
1999-01-01 02:30:00-07:00    0.0
1999-01-01 03:30:00-07:00    0.0
1999-01-01 04:30:00-07:00    0.0
                            ... 
1999-12-31 19:30:00-07:00    0.0
1999-12-31 20:30:00-07:00    0.0
1999-12-31 21:30:00-07:00    0.0
1999-12-31 22:30:00-07:00    0.0
1999-12-31 23:30:00-07:00    0.0
Name: temp_cell, Length: 8760, dtype: float64

In [99]:
dtrange = pd.date_range(start='1999-01-01 00:00:00-07:00', end='1999-12-31 23:50:00-07:00',freq='10T')
new_temp = temp_cell.resample('10T').bfill().reindex(dtrange).bfill().fillna(0)

In [112]:
new_temp.index = new_temp.index.tz_localize(None).tz_localize('UTC') + pd.DateOffset(years=22)
new_temp

2021-01-01 00:00:00+00:00    0.0
2021-01-01 00:10:00+00:00    0.0
2021-01-01 00:20:00+00:00    0.0
2021-01-01 00:30:00+00:00    0.0
2021-01-01 00:40:00+00:00    0.0
                            ... 
2021-12-31 23:10:00+00:00    0.0
2021-12-31 23:20:00+00:00    0.0
2021-12-31 23:30:00+00:00    0.0
2021-12-31 23:40:00+00:00    0.0
2021-12-31 23:50:00+00:00    0.0
Name: temp_cell, Length: 52560, dtype: float64

In [113]:
weather_year = pd.read_csv(os.path.join(TEST_DATA_DIR,'weather_year_pytest.csv'), index_col=0, parse_dates=True)
#weather_year['temp_cell'] = PSM_NEW['temp_cell']
weather_year['temp_cell'] = new_temp
weather_year

,air_temperature,wind_speed,dhi,ghi,dni,relative_humidity,temp_cell
2021-01-01 00:00:00+00:00,-2.3,1.1,0,0,0,56.22,0.0
2021-01-01 00:10:00+00:00,-2.4,1.1,0,0,0,56.64,0.0
2021-01-01 00:20:00+00:00,-2.5,1.1,0,0,0,57.06,0.0
2021-01-01 00:30:00+00:00,-2.5,1.1,0,0,0,57.06,0.0
2021-01-01 00:40:00+00:00,-2.6,1.1,0,0,0,55.94,0.0
...,...,...,...,...,...,...,...
2021-12-31 23:10:00+00:00,-2.0,2.8,19,19,0,71.14,0.0
2021-12-31 23:20:00+00:00,-2.0,2.8,13,13,0,71.14,0.0
2021-12-31 23:30:00+00:00,-2.0,2.8,7,7,0,71.14,0.0
2021-12-31 23:40:00+00:00,-2.0,2.8,0,0,0,71.14,0.0


In [115]:
weather_year.to_csv('weather_year_pytest.csv')